In [3]:
### Loading required modules

import numpy as np
import tensorflow as tf
import os,sys
import datetime
import pickle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img, save_img
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dropout, Activation, Dense
from tensorflow.keras.layers import Flatten, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
import cv2
from matplotlib import pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error, r2_score

tf.config.list_physical_devices('GPU')

path="/home/ubuntu/pediatric_dental_age_prediction/mask/"

print("Loaded all modules")

Loaded all modules


In [2]:
### Synchronized augmentation of the image and its true mask

def syncAug(mode, numAugPerImg):
    datagen_args=dict(rotation_range=5,
                      width_shift_range=0.1,
                      height_shift_range=0.05,
                      zoom_range=0.15,
                      horizontal_flip=True,
                      fill_mode='nearest')
    
    image_datagen = ImageDataGenerator(**datagen_args)
    trueMask_datagen = ImageDataGenerator(**datagen_args)
    
    print("Original image files: {0}".format(path+mode+"_image"))
    print("Saving to ... : {0}".format(path+mode+"_image_aug"))
    
    print("Original true mask files: {0}".format(path+mode+"_trueMask"))
    print("Saving to ... : {0}".format(path+mode+"_trueMask_aug"))
    
    for name_img in os.listdir(path+mode+"_image"):
        img = cv2.imread(path+mode+"_image/{0}".format(name_img))  # cv2 img
        imgRes = cv2.resize(img, dsize=(512, 256)) ### Resizing to 
        imgResArray = img_to_array(imgRes)
        imgResArray = imgResArray.reshape((1,) + imgResArray.shape)
        
        trueMask = cv2.imread(path+mode+"_trueMask/{0}".format(name_img))  # cv2 img
        trueMaskRes = cv2.resize(trueMask, dsize=(512, 256)) ### Resizing to 
        trueMaskResArray = img_to_array(trueMaskRes)
        trueMaskResArray = trueMaskResArray.reshape((1,) + trueMaskResArray.shape)
    
        for i in range(numAugPerImg):
            SEED = np.random.randint(1,10000)
            augImg =image_datagen.flow(imgResArray, seed=SEED).next()[0,:,:,:]
            augTrueMask = trueMask_datagen.flow(trueMaskResArray, seed=SEED).next()[0,:,:,:]
        
            save_img(path+mode+"_image_aug/{0}_0_{1}.jpeg".format(name_img.replace(".png",""),SEED), 
                     augImg, file_format="jpeg")
        
            save_img(path+mode+"_trueMask_aug/{0}_0_{1}.jpeg".format(name_img.replace(".png",""),SEED), 
                     augTrueMask, file_format="jpeg")
            
    print("Data augmentation done for {0} set with {1} per each image and its mask".format(mode, numAugPerImg))

syncAug("train", 20)
syncAug("test", 10)

Original image files: /home/ubuntu/project/mask_test/train_image
Saving to ... : /home/ubuntu/project/mask_test/train_image_aug
Original true mask files: /home/ubuntu/project/mask_test/train_trueMask
Saving to ... : /home/ubuntu/project/mask_test/train_trueMask_aug
Data augmentation done for train set with 20 per each image and its mask
Original image files: /home/ubuntu/project/mask_test/test_image
Saving to ... : /home/ubuntu/project/mask_test/test_image_aug
Original true mask files: /home/ubuntu/project/mask_test/test_trueMask
Saving to ... : /home/ubuntu/project/mask_test/test_trueMask_aug
Data augmentation done for test set with 10 per each image and its mask
